In [1]:
epochs= 25

# SplitNN for Vertically Partitioned Data

Recap: The previous tutorial looked at building a basic SplitNN, where an NN was split into two segments on two seperate hosts. However, what if clients have multi-modal multi-institutional collaboration?

<b>What is Vertically Partitioned Data?</b> Data is said to be vertically partitioned when several organizations own different attributes or modalities of information for the same set of entities.

<b>Why use Partitioned Data?</b> Partition allows for orgnizations holding different modalities of data to learn distributed models without data sharing. Partitioning scheme is traditionally used to reduce the size of data by splitting and distribute to each client.
 
<b>Description</b>This configuration allows for multiple clients holding different modalities of data to learn distributed models without data sharing. As a concrete example we walkthrough the case where radiology centers collaborate with pathology test centers and a server for disease diagnosis. Radiology centers holding imaging data modalities train a partial model upto the cut layer. In the same way the pathology test center having patient test results trains a partial model upto its own cut layer. The outputs at the cut layer from both these centers are then concatenated and sent to the disease diagnosis server that trains the rest of the model. This process is continued back and forth to complete the forward and backward propagations in order to train the distributed deep learning model without sharing each others raw data. In this tutorial, we split a single flatten image into two segments to mimic different modalities of data, you can also split it into arbitrary number.

<img src="images/config_2.png" width="40%">

In this tutorial, we demonstrate the SplitNN architecture with 2 segments[[1](https://arxiv.org/abs/1812.00564)].This time:

- <b>$Client_{1}$</b>
    - Has Model Segment 1
    - Has the handwritten images segment 1
- <b>$Client_{2}$</b>
    - Has model Segment 1
    - Has the handwritten images segment 2
- <b>$Server$</b> 
    - Has Model Segment 2
    - Has the image labels
    
Author:
- Abbas Ismail - github：[@abbas5253](https://github.com/abbas5253)

In [2]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import syft as sy
import numpy as np

hook = sy.TorchHook(torch)

from distribute_data import Distribute_MNIST

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/ab_53/miniconda3/envs/PySyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [3]:
class SplitNN(torch.nn.Module):
    
    def __init__(self, models, optimizers, data_owner, label_owner, server):
        
        self.label_owner = label_owner
        self.data_owners = data_owner
        self.optimizers = optimizers
        self.models = models
        self.server = server
        super().__init__()
        
        
    def forward(self, data_pointer):
        
        #individual client's output upto their respective cut layer
        client_output = {}

        #outputs that is moved to server and subjected to concatenate for server input
        remote_outputs = []

        #iterate over each client and pass thier inputs to respective model segment and move outputs to server
        for owner in self.data_owners:
            client_output[owner.id] = self.models[owner.id][0](data_pointer[owner.id].reshape([-1, 14*28]))
            remote_outputs.append(
                client_output[owner.id].move(self.server).requires_grad_()
            )

        #concat outputs from all clients at server's location
        server_input = torch.cat(remote_outputs, 1)

        #pass concatenated output from server's model segment
        server_output = self.models["server"][0](server_input)
        
        # move to label_owner
        server_output = server_output.move(self.label_owner).requires_grad_()
        
        #pass through the back segment and return prediction
        pred = models[self.label_owner.id][1](server_output)
        
        return pred    
        
    def zero_grads(self):
        for opt in self.optimizers:
            opt.zero_grad()
        
    def step(self):
        for opt in self.optimizers:
            opt.step()
    
    def train(self):
        for loc in self.models.keys():
            for i in range(len(self.models[loc])):
                self.models[loc][i].train()
    
    def eval(self):
        for loc in self.models.keys():
            for i in range(len(self.models[loc])):
                self.models[loc][i].eval()
            
#     @property
#     def location(self):
#         return self.models[0].location if self.models and len(self.models) else None

In [4]:
# Data preprocessing
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

trainset = datasets.MNIST('mnist', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# create some workers
client_1 = sy.VirtualWorker(hook, id="client_1")
client_2 = sy.VirtualWorker(hook, id="client_2")

server = sy.VirtualWorker(hook, id= "server") 

data_owners = (client_1, client_2)
label_owner = client_1
model_locations = [client_1, client_2, server]

#Split each image and send one part to client_1, and other to client_2
distributed_trainloader = Distribute_MNIST(data_owners=data_owners, data_loader=trainloader)

In [15]:
input_size= [28*14, 28*14]
hidden_sizes= {"client_1": [32], "client_2":[32], "server":[64, 128]}

#create model segment for each worker
models = {
        "client_1" : [
                nn.Sequential(
                nn.Linear(input_size[0], hidden_sizes["client_1"][0]),
                nn.ReLU(),
                ),
                
                nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10),
                nn.LogSoftmax(dim=1)
                )
            ],

        "client_2" : [
                nn.Sequential(
                nn.Linear(input_size[1], hidden_sizes["client_2"][0]),
                nn.ReLU(),
                )
            ],
    
    
        "server": [
                nn.Sequential(
                nn.Linear(hidden_sizes["server"][0], hidden_sizes["server"][1]),
                nn.ReLU(),
                ),
            ]
}


# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.05,)
    for location in model_locations
        for model in models[location.id]
            
]


#send model segement to each client and server
for location in model_locations:
    for model in models[location.id]:
        model.send(location)

In [11]:
def train(x, target, splitNN):
    
    #1) Zero our grads
    splitNN.zero_grads()
    
    #2) Make a prediction
    pred = splitNN.forward(x)
  
    #3) Figure out how much we missed by
    criterion = nn.NLLLoss()
    loss = criterion(pred, target.reshape(-1, 64)[0])
    
    #4) Backprop the loss on the end layer
    loss.backward()
    
    #5) Feed Gradients backward through the network
    #splitNN.backward()
    
    #6) Change the weights
    splitNN.step()
    
    return loss.detach().get()

In [16]:
splitnn = SplitNN(models, optimizers, data_owners, label_owner, server)

for i in range(epochs):
    
    running_loss = 0
    splitnn.train()
    for images, labels in distributed_trainloader:
        labels = labels.send(label_owner)
        loss = train(images, labels, splitnn)
        running_loss += loss

    else:
        print("Epoch {} - Training loss: {}".format(i, running_loss/len(trainloader)))

Epoch 0 - Training loss: 2.058415174484253
Epoch 1 - Training loss: 1.3016088008880615
Epoch 2 - Training loss: 0.9639093279838562
Epoch 3 - Training loss: 0.852246880531311
Epoch 4 - Training loss: 0.7964003086090088
Epoch 5 - Training loss: 0.7589868903160095
Epoch 6 - Training loss: 0.7307530641555786
Epoch 7 - Training loss: 0.7083235383033752
Epoch 8 - Training loss: 0.6899858117103577
Epoch 9 - Training loss: 0.6746877431869507
Epoch 10 - Training loss: 0.6616837978363037
Epoch 11 - Training loss: 0.6504472494125366
Epoch 12 - Training loss: 0.6405941843986511
Epoch 13 - Training loss: 0.6318073272705078
Epoch 14 - Training loss: 0.6239115595817566
Epoch 15 - Training loss: 0.6166834831237793
Epoch 16 - Training loss: 0.609996497631073
Epoch 17 - Training loss: 0.6037497520446777
Epoch 18 - Training loss: 0.5978021621704102
Epoch 19 - Training loss: 0.5921556949615479
Epoch 20 - Training loss: 0.5867525339126587
Epoch 21 - Training loss: 0.5815531611442566
Epoch 22 - Training los

In [17]:
def test(model, dataloader, dataset_name):
    correct = 0
    with torch.no_grad():
        for data_ptr, label in dataloader:
            output = splitnn.forward(data_ptr).get()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).sum()

    print("{}: Accuracy {}/{} ({:.0f}%)".format(dataset_name, 
                                                correct,
                                                len(dataloader)* 64, 
                                                100. * correct / (len(dataloader) * 64) ))

In [18]:
#prepare and distribute test dataset
testset = datasets.MNIST('mnist', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
distributed_testloader = Distribute_MNIST(data_owners=data_owners, data_loader=testloader)

#Accuracy on train and test sets
test(models, distributed_trainloader, "Train set")
test(models, distributed_testloader, "Test set")

Train set: Accuracy 49546/59968 (83%)
Test set: Accuracy 8334/9984 (83%)
